In [ ]:
import numpy as np
import pandas as pd
import string
import re
import tensorflow as tf

In [ ]:
english_text=pd.read_table('/content/drive/My Drive/WORK/Seq2Seq model/test_en.en',names=['English'])
hindi_text=pd.read_table('/content/drive/My Drive/WORK/Seq2Seq model/test_hi.hi',names=['Hindi'])

In [ ]:
english_text['English']=english_text['English'].apply(lambda x: x.lower())
hindi_text['Hindi']=hindi_text['Hindi'].apply(lambda x:x.lower())
exclude = set(string.punctuation) # Set of all special characters

# Remove all the special characters
english_text['English']=english_text['English'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
hindi_text['Hindi']=hindi_text['Hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', string.digits)
english_text['English']=english_text['English'].apply(lambda x: x.translate(remove_digits))
hindi_text['Hindi'] = hindi_text['Hindi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
english_text['English']=english_text['English'].apply(lambda x: x.strip())
hindi_text['Hindi']=hindi_text['Hindi'].apply(lambda x: x.strip())
english_text['English']=english_text['English'].apply(lambda x: re.sub(" +", " ", x))
hindi_text['Hindi']=hindi_text['Hindi'].apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
hindi_text['Hindi'] = hindi_text['Hindi'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
english_text.drop(1452,inplace=True)#Missing hindi translation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(english_text.English, hindi_text.Hindi, test_size = 0.2)

In [ ]:
english_words=[]
hindi_words=[]
for line in english_text.English:
    for word in line.split(' '):
        if word not in english_words:
            english_words.append(word)
for line in hindi_text.Hindi:
    for word in line.split(' '):
        if word not in hindi_words:
            hindi_words.append(word)
max_english_sentence=max([len(line.split(' ')) for line in english_text.English ])
max_hindi_sentence=max([len(line.split(' ')) for line in hindi_text.Hindi])

In [ ]:
input_token_index=dict([(word,i) for i,word in enumerate(english_words)])
target_token_index=dict([(word,i) for i,word in enumerate(hindi_words)])
reverse_input_index=dict([(i,word) for i,word in enumerate(english_words)])
reverse_target_index=dict([(i,word) for i,word in enumerate(hindi_words)])

In [ ]:
def generate_batch(X,y,batch_size=128):
    while True:
        for j in range(0,len(X),batch_size):
            encoder_input_data=np.zeros((batch_size,max_english_sentence))
            decoder_input_data=np.zeros((batch_size,max_hindi_sentence))
            decoder_target_data=np.zeros((batch_size,max_hindi_sentence,len(hindi_words)))
            for i,(input_text,target_text) in enumerate(zip(X[j:j+batch_size],y[j:j+batch_size])):
                for t,word in enumerate(input_text.split(' ')):
                    encoder_input_data[i,t]=input_token_index[word]
                for t,word in enumerate(target_text.split(' ')):
                    if t<len(target_text.split(' '))-1:
                        decoder_input_data[i,t]=target_token_index[word]
                    if t>0:
                        decoder_target_data[i,t-1,target_token_index[word]]=1
        yield ([encoder_input_data,decoder_input_data],decoder_target_data)            

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM,Dense,Input,Embedding

In [ ]:
encoder_inputs=Input(shape=(None,))
encoder_embedding=Embedding(len(english_words),128,mask_zero=True)(encoder_inputs)
encoder_lstm=LSTM(128,return_state=True)
encoder_outputs,state_h,state_c=encoder_lstm(encoder_embedding)
encoder_states=[state_h,state_c]

In [ ]:
decoder_inputs=Input(shape=(None,))
decoder_embedding=Embedding(len(hindi_words),128,mask_zero=True)
dec_emb = decoder_embedding(decoder_inputs)
decoder_lstm=LSTM(128,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(dec_emb,initial_state=encoder_states)
decoder_dense=Dense(len(hindi_words),activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

model=Model([encoder_inputs,decoder_inputs],decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100
#checkpoint_path = "eng-hin.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/My Drive/WORK/Seq2Seq model', 
    verbose=1, 
    save_weights_only=True,
    save_freq=5)
model.fit_generator(generator=generate_batch(X_train,y_train,128),
                   steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size,
                    callbacks=[cp_callback])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
 4/15 [=======>......................] - ETA: 59s - loss: 1.8532 - accuracy: 0.0570
Epoch 00001: saving model to /content/drive/My Drive/WORK/Seq2Seq model
 9/15 [=================>............] - ETA: 37s - loss: 1.7121 - accuracy: 0.0507
Epoch 00001: saving model to /content/drive/My Drive/WORK/Seq2Seq model
14/15 [===========================>..] - ETA: 6s - loss: 1.5796 - accuracy: 0.0488 
Epoch 00001: saving model to /content/drive/My Drive/WORK/Seq2Seq model
15/15 [==============================] - 107s 7s/step - loss: 1.5599 - accuracy: 0.0486 - val_loss: 2.0207 - val_accuracy: 0.0543
Epoch 2/100
 4/15 [=======>......................] - ETA: 58s - loss: 1.2594 - accuracy: 0.0456
Epoch 00002: saving model to /content/drive/My Drive/WORK/Seq2Seq model
 9/15 [=================>............] - ETA: 37s - loss: 1.2454 - accuracy: 0.0456
Epoch 00002: saving model to /content/drive/My Drive/WORK/Seq2

In [ ]:
model.save_weights("eng-hin2.h5")
#model.load_weights('eng-hin.h5')

In [ ]:
encoder_model=Model(encoder_inputs,encoder_states)
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = decoder_embedding(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_index[sampled_token_index]
        decoded_sentence +=' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence[:-4]

In [ ]:
input_data=np.zeros((1,max_english_sentence))
text="i am not talking about nsa"
for i,word in enumerate(text.split(' ')):
  input_data[0,i]=input_token_index[word]

In [ ]:
input_token_index['good']     

1138

In [ ]:
decode_sequence(input_data)

' एनएसए के लिए निर्णय लेने में सक्षम बनाता है। '